# Hate Speech Classifier
Classifies Tweets into (0 = hate speech, 1 = offensive language, 2 = neither) using a convolutional neural network trained on https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset.

Accuracy: ~87%

In order to handle false positives, sentiment analysis is used in addition to this model.

In [ ]:
!pip install -q demoji
!pip install -q contractions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import demoji
import contractions
import string
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, concatenate
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
import pickle

### Tweet Preprocessor

In [ ]:
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)

stop_words = set(stopwords.words('english') + ["rt"])
lemmatizer = WordNetLemmatizer()

abbreviations = {
    "lol": "laughing",
    "lmao": "laughing",
    "lmfao": "laughing",
    "kys": "kill yourself",
    "stfu": "shut up",
    "lmk": "let me know",
    "nvm": "nevermind",
    "tbh": "to be honest",
    "bc": "because",
    "cuz": "because",
    "btw": "by the way",
    "cya": "see you",
    "imho": "in my humble opinion",
    "imo": "in my opinion",
    "ftw": "for the win",
    "fyi": "for your information",
    "gtg": "got to go",
    "gr8": "great",
    "ily": "i love you",
    "fs": "for sure",
    "jk": "just kidding",
    "ong": "for real",
    "fr": "for real",
    "thx": "thanks",
    "omg": "oh my god",
    "ttyl": "talk to you later",
}


def twitter_preprocess(text: str):
    """
    Preprocessor for tweets
    Remove punctuation
    Remove hashtags
    Remove stopwords
    Remove URLs, hashtags, and usernames
    Expand common abbreviations and contractions
    Convert emojis to words
    Convert to lowercase
    Remove non-ASCII characters
    Stem words
    """
    tokens = tweet_tokenizer.tokenize(text)
    filtered_tokens = []
    for token in tokens:
        # Remove punctuation
        if token in string.punctuation:
            continue
        # Remove hashtags
        if token.startswith('#'):
            token = token[1:]
        # Remove stopwords
        if token in stop_words:
            continue
        # Remove urls
        if token.startswith('http'):
            continue
        # Expand common abbreviations and contractions
        token = contractions.fix(token)
        for key, value in abbreviations.items():
            token = re.sub(r'\b' + key + r'\b', value, token)
        # Convert emojis to words
        token = demoji.replace_with_desc(token, '')
        # Remove non-ASCII characters
        token = re.sub(r'[^\x00-\x7F]+', '', token)
        # Lemmatize
        token = lemmatizer.lemmatize(token)
        if token != '':
            filtered_tokens.append(token)
    text = " ".join(filtered_tokens)
    return text

### Load Dataset

In [ ]:
df = pd.read_csv('./data/labeled_data.csv')
df = pd.concat([df['tweet'],df['class']], axis=1)
df['tweet'] = df['tweet'].apply(twitter_preprocess)
X = df['tweet'].fillna("Invalid").values
y = df['class'].values
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
max_features = 20000
maxlen = 100

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

# Export tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

y_train = to_categorical(y_train, num_classes=label_count)
y_test = to_categorical(y_test, num_classes=label_count)

hate, offensive, neither = np.bincount(df['class'])
total = hate + offensive + neither

class_weights = {0: (1 / hate)*(total)/3.0, 1: (1 / offensive)*(total)/3.0, 2: (1 / neither)*(total)/3.0}
class_weights

### Create Model

In [ ]:
label_count = 3

def build_model(conv_layers = 2, max_dilation_rate = 4):
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.3)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt_x = Conv1D(2*embed_size, kernel_size = 3)(x)
    out_conv = []

    for dilation_rate in range(max_dilation_rate):
        x = prefilt_x
        for i in range(3):
            x = Conv1D(32*2**(i), kernel_size = 3, dilation_rate = 2**dilation_rate)(x)    
        out_conv += [Dropout(0.3)(GlobalMaxPooling1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(label_count, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = build_model()

### Train Model

In [ ]:
file_path="hatespeech.bin"
# Export best version of model
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)

model.fit(X_train, y_train, 
          batch_size=512,  
          epochs=15,
          callbacks=[checkpoint, early],
          validation_data=(X_test, y_test),
          class_weight=class_weights
         )

### Evaluate Model

In [ ]:
model.load_weights(file_path)
model.evaluate(X_test, y_test)